# Data Processing
 - Add a new column of Orientation_trans
 - Generate a full joint list that contains both years of ili

In [ ]:
# convert the Orientation from degrees to length of distance to startpoint/ arc length
data["Orientation_trans"]=math.pi*pipe_diam*(data["Orientation"]/360)


full_joint_list = []
# Filter Criteria: must have both years of inspection
for joint in data["Joints"]:
    if(any(data.loc[data["Joints"]==joint,"iliyr"] ==fix_iliyr) & any(data.loc[data["Joints"]==joint,"iliyr"] ==move_iliyr)):
        if joint in full_joint_list:
            continue
        full_joint_list = numpy.append(full_joint_list, joint)

full_joint_list = full_joint_list.astype("int")
  # keep two columns: distance to us weld and orientation(m)




In [ ]:
# print the list
print(full_joint_list)

# Prepare the Evaluation Table

In [ ]:
eval_table = ["Joint", "Fix Size","Move Size","Ratio(Fix/Move)","Match Rate(Fix)", "Match Rate(Move)","Run Time"]

with open('Evaluation Table_%s.csv'%move_iliyr,'a') as f:
    writer = csv.writer(f)
    writer.writerow(eval_table)

# Install the Engine for Exporting Images from the Main Script

In [ ]:
# required for exporting the images created under plotly!
!pip install kaleido

# Prepare the data and call the Main Script

- For each joint, convert fix data and move data to lists
- Call the main script / 3rd layer of the program

In [ ]:
# record the start time of the whole project
start_time_project = time.time()

counter = 1

# loop over the joint number and execute the main script
for joint in full_joint_list[counter:]:
    print(joint)
    # fix data of a single joint: two columns of DS_TO_US and Orientation
    fix_data = data.loc[(data["Joints"]==joint)&(data["iliyr"]==fix_iliyr), ["DS_TO_US","Orientation_trans"]] # 240
    # current Copy_of_Clean_v1_Lin_Guo.ipynb requires list input of the datset
    fix_data = fix_data.values.tolist()
    # move data of a single joint: two columns of DS_TO_US and Orientation
    move_data = data.loc[(data["Joints"]==joint)&(data["iliyr"]==move_iliyr), ["DS_TO_US","Orientation_trans"]] # 203
    # move data must have at leats two features for the algorithm to run successfully 
    if move_data.shape[0]<=2:
        continue
    # current Copy_of_Clean_v1_Lin_Guo.ipynb requires list input of the datset
    move_data = move_data.values.tolist()
    
    # for the datasets run the main execution script
    try:
        %run Copy_of_Clean_v1_Lin_Guo.ipynb
    
    except Exception as error_msg:
        # record the error message
        with open('Evaluation Table_%s.csv'%move_iliyr,'a') as f:
            writer = csv.writer(f)
            writer.writerow([joint, error_msg])
        # skip to the next loop and proceed on to the next joint
        pass
    try:
        x_trans = pd.read_csv("x_trans_%s.csv"%joint, header=None)
        #update the merged list and format the output'
        data.loc[(data["Joints"]==joint)&(data["iliyr"]==move_iliyr),"DS_TO_US_trans"]=x_trans.iloc[:,0].values
        data.loc[(data["Joints"]==joint)&(data["iliyr"]==fix_iliyr),"DS_TO_US_trans"]=data.loc[(data["Joints"]==joint)&(data["iliyr"]==fix_iliyr),"DS_TO_US"]
    # if put the except behind x_trans, then chances are there's no update for some joints, and then there will be mis-alignment. 
    except:
        pass
    # record how many joints have been tested
    counter += 1

end_time_project = time.time()


In [ ]:
#convert the runtime for the whole project to minutes
runtime_project = (end_time_project-start_time_project)/60
# print the runtime in 2 digits
runtime_project = "%.2f"%runtime_project

# the runtime for each iliyr
print("The runtime for the whole project is "+runtime_project)
print("The total number of joints under testing is %s"%counter)